In [2]:
from langchain_huggingface import HuggingFaceEndpoint, ChatHuggingFace
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from constants import STARTING_FEN 
import chess

In [ ]:
# using deepseek distilled r1 (deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B)

llm = HuggingFaceEndpoint(
    repo_id="microsoft/phi-2",
    # huggingfacehub_api_token="",
    task="text-generation",
    do_sample=False,
)

chat = ChatHuggingFace(llm=llm, verbose=True)




In [4]:

template = PromptTemplate(
    input_variables=["fen", "player", "bot"],
    template='''We are playing a chess game (me vs you). I am {player}, and you are {bot}. It is your turn to move. Respond with your move in standard chess notation (e.g., 'e4') followed by the FULL FEN string after your move. The current FEN is: "{fen}"
    Format your response exactly as follows:
    Move: [your move]
    FEN: [resulting FEN string]
    '''
    # template="We are playing a chess game. I am {player}, you are {bot}. Make a move and respond with the FULL FEN string resulting from your move. The current FEN is {fen}"
)

invoked = template.invoke(
    {"fen": f"{STARTING_FEN}",
     "player": "black",
     "bot": "white"}
)


In [5]:
print(invoked.to_string())

We are playing a chess game (me vs you). I am black, and you are white. It is your turn to move. Respond with your move in standard chess notation (e.g., 'e4') followed by the FULL FEN string after your move. The current FEN is: "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
    Format your response exactly as follows:
    Move: [your move]
    FEN: [resulting FEN string]
    


In [6]:
test='''
Move: e4
FEN: rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1
'''
test.split('FEN: ')[-1].strip()

'rnbqkbnr/pppppppp/8/8/4P3/8/PPPP1PPP/RNBQKBNR b KQkq e3 0 1'

In [11]:
cc = None
try:
    resp = llm.invoke(invoked)
    valid = chess.Board(resp.split('FEN: ')[-1].strip()).is_valid()
    print(resp)
except Exception as e:
    print(f"Model unavailable\n{e}")
    print(type(e))
    cc = e

/Users/rodericklan/Projects/ChessVLM-backend/.venv/lib/python3.13/site-packages/huggingface_hub/utils/_deprecation.py:131: FutureWarning: 'post' (from 'huggingface_hub.inference._client') is deprecated and will be removed from version '0.31.0'. Making direct POST requests to the inference server is not supported anymore. Please use task methods instead (e.g. `InferenceClient.chat_completion`). If your use case is not supported, please open an issue in https://github.com/huggingface/huggingface_hub.
  warnings.warn(warning_message, FutureWarning)


Model unavailable
503 Server Error: Service Temporarily Unavailable for url: https://router.huggingface.co/hf-inference/models/microsoft/phi-2
<class 'huggingface_hub.errors.HfHubHTTPError'>


In [8]:
cc.response.status_code

503

In [9]:
"rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"
"rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1"

'rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1'